In [ ]:
!pip install -q scikit-multiflow

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from skmultiflow.drift_detection import ADWIN, DDM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from matplotlib.colors import ListedColormap
from multiprocessing.pool import ThreadPool
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
from contextlib import suppress
from collections import Counter
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from random import shuffle
import seaborn as sns
from time import time
import pandas as pd
import numpy as np
import warnings
import scipy.io
import pickle
import sys
import gc
import os
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data'
code_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/Codes/Shared Codes'
results_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/results'
feature_selection_results = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/feature_selection_results'
feature_selection_results_evolving = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/feature_selection_results_evolving'

In [ ]:
sys.path.insert(0,code_path)
from genetic_programming import SymbolicRegressor
from binirizer import CustomLabelBinirizer
from ensemble import Ensemble, Classifier
from oselm import OSELMClassifier,set_use_know
from DynamicFeatureSelection import dynamic_feature_selection
from SharedFunctions import prepare_data,train_and_test,feature_evolving,save_pickle,load_pickle,save_object,load_object,generate_new_samples

In [ ]:
filenames = ['kddcup99_csv.csv','ISCX2012.csv','CSE-CIC2018.csv','CICIDS2017.csv','7recurrentDrift.csv', 'blip.csv', 'incrementalDrift.csv',
             '7gradualDrift.csv', '7suddenDrift.csv']
filenames = list(map(lambda x: os.path.join(data_path, x), filenames))
filenames

['/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/kddcup99_csv.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/ISCX2012.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/CSE-CIC2018.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/CICIDS2017.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/7recurrentDrift.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/blip.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/incrementalDrift.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/7gradualDrift.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/7suddenDrift.csv']

In [ ]:
def genetic_programming():
    return SymbolicRegressor(population_size=10,
            generations=5, stopping_criteria=0.85,
            p_crossover=0.7, p_subtree_mutation=0.1,
            p_hoist_mutation=0.05, p_point_mutation=0.1,
            max_samples=0.7, verbose=1,
            parsimony_coefficient=1e-4, random_state=42,
            function_set=['avg2', 'avg3', 'avg5',
                          'median3', 'median5', 'maximum2', 'maximum3', 'maximum5'],
            metric='f1-score')

In [ ]:
def generate_oselm_models(number_of_hidden_neurons, apply_model_replacement=False):
    models= [OSELMClassifier(number_of_hidden_neurons, 'relu', binarizer=CustomLabelBinirizer(), random_state=42),
             OSELMClassifier(number_of_hidden_neurons, 'relu', binarizer=CustomLabelBinirizer(), random_state=42),
             OSELMClassifier(number_of_hidden_neurons, 'relu', binarizer=CustomLabelBinirizer(), random_state=42),
             OSELMClassifier(number_of_hidden_neurons, 'relu', binarizer=CustomLabelBinirizer(), random_state=42),
             ]

    ensemble = Ensemble(classifiers=models, program=genetic_programming(), apply_model_replacement=apply_model_replacement)
    return ensemble

def generate_ml_models(number_of_hidden_neurons, apply_model_replacement=False):
    models = [
              KNeighborsClassifier(5),
              KNeighborsClassifier(5),
              # DecisionTreeClassifier(),
              LogisticRegression(),
              LogisticRegression(),
              GaussianNB(),
              GaussianNB(),
              GaussianNB(),
              ]
    ensemble = Ensemble(classifiers=models, program=genetic_programming(), apply_model_replacement=apply_model_replacement)
    return ensemble

In [ ]:
def concept_drift_detection(drift_detection_obj, sample) -> bool:
    """
    Detect concept drift
    :param drift_detection_obj: sklearn drift detection object (ADWIN, DDM, )
    :param smaple : new instanece of data stream
    return True if concept drift was detected otherwise false
    """
    drift_detection_obj.add_element(sample)
    return drift_detection_obj.detected_change()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
def random_forest_feature_selection(X, y):
    """
    return best feature from X using random forest
    """
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 20))
    sel.fit(X, y)
    return sel.get_support()

In [ ]:
def E2SC4ID (X,
             y,
             sample_index:int,
             buffer:list,
             ensemble: Ensemble,
             drift_detection_obj,
             maxC=8,
             n=2000,
             train_size=0.5,
             drift=False,
             unselected_features=None,
             drift_location={}):
    y_pred = ensemble.global_support_degree(X)
    # if the sample is labeled then insert it into buffer
    if y is not None:
        buffer.append((X, y))
        actual_drift = concept_drift_detection(drift_detection_obj, int(y!=y_pred))
        if actual_drift and not drift:
          drift_location[sample_index] = 'drift'
        drift = drift or actual_drift
        if len(buffer) >= n:
            if drift:
                drift = False
                drift_detection_obj.reset()
                x_buffer, y_buffer = [], []
                for tup in buffer:
                    x_buffer.append(tup[0])
                    y_buffer.append(tup[1])
                ######################################################
                train_size = int(len(x_buffer)*train_size)
                X_train = x_buffer[:train_size]
                y_train = y_buffer[:train_size]
                X_valid = x_buffer[train_size:]
                y_valid = y_buffer[train_size:]
                ######################################################
                __sum = np.array(y_train).sum()
                if 0 ==  __sum or __sum == len(y_train):
                  y_train[0] = 0 if y_train[0] == 1 else 1
                new_models = ensemble.classifier_induction([
                                        model.clf for model in ensemble.classifiers],
                                        X_train,
                                        y_train,
                                        unselected_features)
                if len(ensemble.classifiers) > maxC:
                    ensemble.model_replacement('time')
                ######################################################
                ensemble.update_program(X_valid, y_valid)
            else:
                buffer.clear()
        return ensemble, buffer, drift, drift_location

In [ ]:
def E2SC4ID_STREAM(ensemble, stremdata, y, unselected_features, drift_location, chunk_number,result_save_path_data,key,sample_number=0):
    if not ensemble.fitted:
      y[0] = 0 if y[0] == 1 else 1
      ensemble.fit(stremdata[:200], y[:200])

    drift_detection_obj, drift, buffer = DDM(),False,[]
    if sample_number != 0 :
      drift_location = load_pickle(os.path.join(result_save_path_data, "{}_drift_location.pkl".format(key)))
      drift = load_pickle(os.path.join(result_save_path_data, "{}_drift.pkl".format(key)))
      buffer = load_pickle(os.path.join(result_save_path_data, "{}_buffer.pkl".format(key)))
      ensemble = load_pickle(os.path.join(result_save_path_data, "{}_ensemble.pkl".format(key)))

    for i in tqdm(range(200+sample_number, len(stremdata))):
        X, y_true = stremdata[i], y[i]
        ensemble, buffer, drift, drift_location = E2SC4ID (X,
                                                           y_true,
                                                           sample_index=(i +(chunk_number * 10000)),
                                                           buffer=buffer,
                                                           ensemble=ensemble,
                                                           drift_detection_obj=drift_detection_obj,
                                                           maxC=8,
                                                           n=len(stremdata)-200,
                                                           train_size=0.7,
                                                           drift=drift,
                                                           unselected_features=unselected_features,
                                                           drift_location=drift_location)
        if len(buffer) >= (len(stremdata)-200):
          save_pickle(ensemble, os.path.join(result_save_path_data, "{}_ensemble.pkl".format(key)))
          save_pickle(drift_location, os.path.join(result_save_path_data, "{}_drift_location.pkl".format(key)))
          save_pickle(drift, os.path.join(result_save_path_data, "{}_drift.pkl".format(key)))
          save_pickle(buffer, os.path.join(result_save_path_data, "{}_buffer.pkl".format(key)))
    return ensemble, drift_location

In [ ]:
def main(f_name, generate_model, train_size=3000,apply_model_replacement=False, transfer_learning=False, feature_selection="random_forest", result_save_path="",ChunkNumber=0,sample_number=0,DFS_results_path=''):
  """
  datasets_paths: list of paths (absolute path for each dataset)
  generate_model: function to generate machine learning model.
  train_size: number of samples to be used in training phase.
  transfer_learning: to determine using of transfer learning in the training phase.
  """
  # load the dataset and then process it
  datasets = {}
  first_chunk=True
  d = prepare_data(f_name)
  d = d.sample(frac=1, random_state=42)
  buffer = d.sample(n=5000)
  d.reset_index(inplace=True)
  d.replace([np.inf], 0, inplace=True)
  datasets[f_name.split('/')[-1]] = d
  results = {}
  drift_locations_in_all_dataset = {}
  ensemble = None ####
  ########################################
  for key in tqdm(datasets.keys()):
      result_save_path_data = os.path.join(result_save_path, key)
      drift_location = {} if ChunkNumber==0 else load_pickle(os.path.join(result_save_path_data, "{}_drift_location.pkl".format(key)))
      results[key] = {'model_result': []}
      # convert dataset from dataframe to numpy array.
      data = datasets[key].values
      # split the data into features array and target array.
      X, Y = data[:, 0:-1], data[:, -1].astype('int')
      if not os.path.exists("{}_evolving_matrix.pkl".format(key)):
        a2 = np.random.randint(low=0, high=X.shape[1], size = X.shape[1] // 6).tolist()
        a3 = np.random.randint(low=0, high=X.shape[1], size = X.shape[1] // 5).tolist()
        a4 = np.random.randint(low=0, high=X.shape[1], size = X.shape[1] // 4).tolist()
        evolving_matrix = [a2, a3, a4]
        save_pickle(evolving_matrix, "{}_evolving_matrix.pkl".format(key))
      else:
        evolving_matrix = load_pickle("{}_evolving_matrix.pkl".format(key))
      ensemble = generate_model(number_of_hidden_neurons=X.shape[1]*3 // 2, apply_model_replacement=apply_model_replacement)
      # split the data into chunks (10 chunks)
      chunks_features = np.array_split(X, 10)
      chunks_labels = np.array_split(Y, 10)

      ################# train on each chunk ####################
      print("===================== dataset : {} ======================".format(key))
      chunk_number = 1
      if ChunkNumber>0:
        results = load_pickle(os.path.join(result_save_path_data,"{}_results.pkl".format(key)))
        drift_locations_in_all_dataset = load_pickle(os.path.join(result_save_path_data,"{}_drift_locations_in_all_dataset.pkl".format(key)))
        ensemble = load_object("{}_ensemble".format(key),result_save_path_data)

      for CN,chunk_X, chunk_Y in tqdm(zip([*range(len(chunks_labels))],chunks_features, chunks_labels)):
          if ChunkNumber > CN:
            print("Skip Chunk Number : {}".format(CN))
            continue
          if not first_chunk:
            sample_number=0
            first_chunk = False
          else:first_chunk = False
          try:
            chunk_X, chunk_Y = SMOTE().fit_resample(chunk_X, chunk_Y)
          except:
            if chunk_Y.sum() in [0, 1]:
              new_samples, new_labels = generate_new_samples(buffer, chunk_Y)
              chunk_X = np.concatenate((chunk_X, new_samples))
              chunk_Y = np.concatenate((chunk_Y, new_labels))
          gc.collect()
          unselected_feautres = None
          selected = None
          X_train, X_test, y_train, y_test = chunk_X[:train_size], chunk_X[train_size:], chunk_Y[:train_size], chunk_Y[train_size:]
          if feature_selection[0] == "feature_evolving":
            if feature_selection[1] == "random_forest":
              unselected_feautres = feature_evolving(evolving_matrix=evolving_matrix)
              X_train = np.delete(X_train, unselected_feautres, 1)
              X_test = np.delete(X_test, unselected_feautres, 1)
              selected = random_forest_feature_selection(X_train, y_train)
              unselected_feautres = np.where(selected != 1)[0]
            elif feature_selection[1] == "DFS_feature_selection":
              # selected = dynamic_feature_selection(chunk_X, chunk_Y)
              softmax_results = load_object('softmax_results',DFS_results_path)
              average_results = load_object('average_results',DFS_results_path)
              single_agent_results = load_object('single_agent_results',DFS_results_path)
              random_forest_results = load_object('random_forest_results',DFS_results_path)
              voting_results = load_object('voting_results',DFS_results_path)
              f1_score_for_all_algorithm = [softmax_results['f1'],average_results['f1'],single_agent_results['f1'],random_forest_results['f1'],voting_results['f1']]
              masks = [load_object("softmax_mask_"+str(CN),DFS_results_path),
                        load_object("average_mask_"+str(CN),DFS_results_path),
                        load_object("single_agent_mask_"+str(CN),DFS_results_path),
                        load_object("random_forest_mask_"+str(CN),DFS_results_path),
                        load_object("voting_mask_"+str(CN),DFS_results_path)]
              selected = masks[f1_score_for_all_algorithm.index(max(f1_score_for_all_algorithm))]
              unselected_feautres = np.where(selected != 1)[0]
          else:
            if feature_selection[1] == "random_forest":
              selected = random_forest_feature_selection(X_train, y_train)
              unselected_feautres = np.where(selected != 1)[0]
            elif feature_selection[1] == "DFS_feature_selection":
              # selected = dynamic_feature_selection(X_train, y_train)
              softmax_results = load_object('softmax_results',DFS_results_path)
              average_results = load_object('average_results',DFS_results_path)
              single_agent_results = load_object('single_agent_results',DFS_results_path)
              random_forest_results = load_object('random_forest_results',DFS_results_path)
              voting_results = load_object('voting_results',DFS_results_path)
              f1_score_for_all_algorithm = [softmax_results['f1'][CN],average_results['f1'][CN],single_agent_results['f1'][CN],random_forest_results['f1'][CN],voting_results['f1'][CN]]
              masks = [load_object("softmax_mask_"+str(CN),DFS_results_path),
                        load_object("average_mask_"+str(CN),DFS_results_path),
                        load_object("single_agent_mask_"+str(CN),DFS_results_path),
                        load_object("random_forest_mask_"+str(CN),DFS_results_path),
                        load_object("voting_mask_"+str(CN),DFS_results_path)]
              selected = masks[f1_score_for_all_algorithm.index(max(f1_score_for_all_algorithm))]
              unselected_feautres = np.where(selected != 1)[0]

          if not os.path.exists(result_save_path_data):
            os.mkdir(result_save_path_data)
          if transfer_learning:
            temp = np.squeeze(X_train[:, selected]) if len(list(X_train[:, selected].shape))>2 else X_train[:, selected]
            ensemble, drift_location = E2SC4ID_STREAM(ensemble=ensemble, stremdata=temp, y=y_train, unselected_features=None, drift_location=drift_location,
                                                      chunk_number=chunk_number, result_save_path_data=result_save_path_data,key=key,sample_number=sample_number)
            temp = np.squeeze(X_test[:, selected]) if len(list(X_test[:, selected].shape))>2 else X_test[:, selected]
            ensemble.evaluate(temp, y_test, chunk_number)
          else:
            ensemble, drift_location = E2SC4ID_STREAM(ensemble=generate_model(number_of_hidden_neurons=X.shape[1]*3 // 2,apply_model_replacement=apply_model_replacement),
                                                      stremdata=X_train, y=y_train, unselected_features=unselected_feautres, drift_location=drift_location,
                                                      chunk_number=chunk_number,result_save_path_data=result_save_path_data,key=key,sample_number=sample_number)
            ensemble.evaluate(X_test, y_test, chunk_number)
          save_pickle(drift_location, os.path.join(result_save_path_data, "{}_drift_location.pkl".format(key)))

          temp = np.squeeze(X_test[:, selected]) if len(list(X_test[:, selected].shape))>2 else X_test[:, selected]
          y_pre = ensemble.predict(temp)
          results[key][chunk_number] = {"y_true" : y_test, "y_pred": y_pre}
          results[key]['model_result'].append(ensemble.scores)
          if transfer_learning:
             ensemble.fit(temp, y_test, None)
          chunk_number += 1
          drift_locations_in_all_dataset[key] = drift_location

          save_pickle(ensemble, os.path.join(result_save_path_data, "{}_ensemble.pkl".format(key)))
          save_pickle(results, os.path.join(result_save_path_data, "{}_results.pkl".format(key)))
          save_pickle(drift_locations_in_all_dataset, os.path.join(result_save_path_data, "{}_drift_locations_in_all_dataset.pkl".format(key)))

In [ ]:
data_name = ['kddcup99','ISCX2012','CSE-CIC2018','CICIDS2017','7recurrentDrift', 'blip', 'incrementalDrift','7gradualDrift', '7suddenDrift']

In [ ]:
# # use_know = False
# # set_use_know(use_know)
# # path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
# # os.makedirs(path, exist_ok=True)
# # for f_name,d_name in zip(filenames,data_name):
# #   DFS_results_path = os.path.join(feature_selection_results_evolving,d_name)
# #   main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
# #        feature_selection=["feature_evolving",'DFS_feature_selection'], result_save_path=path,DFS_results_path=DFS_results_path)

In [ ]:
DATA_NUMBER=0
ChunkNumber=0
sample_number=sum([0])
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results_evolving,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'],result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber,sample_number=sample_number)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : kddcup99_csv.csv ======================


0it [00:00, ?it/s]

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   128.70                1       81                1                1      0.53s


  0%|          | 0/2800 [00:00<?, ?it/s]

...

{8: {'accuracy': 0.9430147058823529, 'precision': 0.8476454293628809, 'recall': 0.9973924380704041, 'f1-score': 0.9164420485175201, 'auc': 0.957797944198795}}
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   128.70          0.99422       81          0.99422          0.99422      0.49s


  0%|          | 0/2800 [00:00<?, ?it/s]

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    66.10         0.986334       73         0.986842         0.986842      0.43s
{9: {'accuracy': 0.9331561733442355, 'precision': 0.8247978436657682, 'recall': 0.9986945169712794, 'f1-score': 0.9034543844109832, 'auc': 0.9509841632475445}}
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   128.70         0.993789       81         0.993789         0.993789      0.47s


  0%|          | 0/2800 [00:00<?, ?it/s]

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    66.10          0.99634        3          0.99701          0.99701      0.42s
{10: {'accuracy': 0.9916492693110647, 'precision': 0.9830287206266318, 'recall': 0.9907894736842106, 'f1-score': 0.9868938401048493, 'auc': 0.9914192016739095}}


In [ ]:
DATA_NUMBER=1
ChunkNumber=0
sample_number=sum([0])
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results_evolving,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'],result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber,sample_number=sample_number)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : ISCX2012.csv ======================


0it [00:00, ?it/s]

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   128.70         0.847458       81         0.847458         0.847458      0.21s
   1   672.00         0.847458     2437         0.847458         0.847458      0.53s
   2   187.60         0.847458      297         0.847458         0.847458      0.12s
   3   779.50         0.847458        9         0.847458         0.847458      0.18s
   4   183.40         0.847458      789         0.847458         0.847458      0.00s


  0%|          | 0/2800 [00:00<?, ?it/s]

...

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    66.10              0.8       73              0.8              0.8      0.50s
   1   539.10              0.8     2234              0.8              0.8      1.08s
   2   184.80              0.8      525              0.8              0.8      0.29s
   3   451.70              0.8       12              0.8              0.8      0.33s
   4   141.60              0.8      750              0.8              0.8      0.00s
{8: {'accuracy': 0.8314696485623003, 'precision': 0.9976232917409388, 'recall': 0.666534339023422, 'f1-score': 0.7991432651118515, 'auc': 0.8324636339552627}}
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Leng

  0%|          | 0/2800 [00:00<?, ?it/s]

{9: {'accuracy': 0.8594783962631374, 'precision': 0.994028230184582, 'recall': 0.7202989771833203, 'f1-score': 0.8353102189781022, 'auc': 0.8580308445238635}}
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   128.70         0.777778       81         0.777778         0.777778      0.20s
   1   672.00         0.777778     2437         0.777778         0.777778      0.49s
   2   187.60         0.777778      297         0.777778         0.777778      0.17s
   3   779.50         0.777778        9         0.777778         0.777778      0.18s
   4   183.40         0.777778      789         0.777778         0.777778      0.00s


  0%|          | 0/2800 [00:00<?, ?it/s]

{10: {'accuracy': 0.8365872705974229, 'precision': 0.9938786867000556, 'recall': 0.6837672281776417, 'f1-score': 0.8101610342481288, 'auc': 0.8396923790290599}}


In [ ]:
DATA_NUMBER=2
ChunkNumber=0
sample_number=sum([0])
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results_evolving,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber,sample_number=sample_number)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : CSE-CIC2018.csv ======================


0it [00:00, ?it/s]

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   128.70         0.997494       81         0.997494         0.997494      0.49s


  0%|          | 0/2800 [00:00<?, ?it/s]

...

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    66.10         0.999762       73                1                1      0.30s
{8: {'accuracy': 0.9224842040376021, 'precision': 0.8323053842307051, 'recall': 1.0, 'f1-score': 0.9084788937409025, 'auc': 0.9370068879148403}}
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   128.70         0.997494       81         0.997494         0.997494      0.47s


  0%|          | 0/2800 [00:00<?, ?it/s]

{9: {'accuracy': 0.7624594969912051, 'precision': 0.6223220012828736, 'recall': 0.9729241877256317, 'f1-score': 0.7590955324309522, 'auc': 0.8019084328798671}}
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   128.70         0.997481       81         0.997481         0.997481      0.47s


  0%|          | 0/2800 [00:00<?, ?it/s]

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    66.10         0.998869        8         0.999404         0.999404      0.34s
{10: {'accuracy': 0.8593870321885108, 'precision': 0.733470770136946, 'recall': 0.9967980788473084, 'f1-score': 0.8450967085171361, 'auc': 0.8851182783772154}}


In [ ]:
DATA_NUMBER=3
ChunkNumber=0
sample_number=sum([0])
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results_evolving,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber,sample_number=sample_number)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : CICIDS2017.csv ======================


0it [00:00, ?it/s]

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   128.70                1       81                1                1      0.64s


  0%|          | 0/2800 [00:00<?, ?it/s]

...

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    66.10         0.958179       73         0.975558         0.975558      0.31s
{2: {'accuracy': 0.9625041848008035, 'precision': 0.939209726443769, 'recall': 0.9844247787610619, 'f1-score': 0.9612858624265469, 'auc': 0.9636318876339448}}
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   128.70         0.990741       81         0.990741         0.990741      0.51s


  0%|          | 0/2800 [00:00<?, ?it/s]

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    66.10          0.97039        7         0.977459         0.977459      0.32s
{3: {'accuracy': 0.9702262142381903, 'precision': 0.9508759876331158, 'recall': 0.9871611982881597, 'f1-score': 0.968678915135608, 'auc': 0.9712925692188928}}
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   128.70         0.991304       81         0.991304         0.991304      0.48s


  0%|          | 0/2800 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
DATA_NUMBER=4
ChunkNumber=0
sample_number=sum([0])
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results_evolving,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber,sample_number=sample_number)

In [ ]:
DATA_NUMBER=5
ChunkNumber=0
sample_number=sum([0])
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results_evolving,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber,sample_number=sample_number)

In [ ]:
DATA_NUMBER=6
ChunkNumber=0
sample_number=sum([0])
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results_evolving,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber,sample_number=sample_number)

In [ ]:
DATA_NUMBER=7
ChunkNumber=0
sample_number=sum([0])
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results_evolving,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber,sample_number=sample_number)

In [ ]:
DATA_NUMBER=8
ChunkNumber=0
sample_number=sum([0])
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results_evolving,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=False,
     feature_selection=["feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber,sample_number=sample_number)